In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import cv2

In [5]:
train_gen = ImageDataGenerator(rescale=1/255.,rotation_range=0.2,zoom_range=0.2,
                              shear_range=0.2,width_shift_range=0.2,height_shift_range=0.3,
                              horizontal_flip=True,vertical_flip=True)
test_gen = ImageDataGenerator(rescale=1/255.,rotation_range=0.2,zoom_range=0.2,
                              shear_range=0.2,width_shift_range=0.2,height_shift_range=0.3,
                              horizontal_flip=True,vertical_flip=True)
train_dir = "Leaf_dataset/train/"
test_dir = "Leaf_dataset/test/"

train_data = train_gen.flow_from_directory(train_dir,target_size=(224,224),batch_size=32,
                                          class_mode="categorical")
test_data = test_gen.flow_from_directory(test_dir,target_size=(224,224),batch_size=32,
                                         class_mode="categorical")

Found 77213 images belonging to 46 classes.
Found 19237 images belonging to 46 classes.


In [7]:
path = pathlib.Path("Leaf_dataset/train/")
class_name = np.array([item.name for item in path.glob("*")])
class_name

array(['Apple___Apple_scab', 'Apple___Black_rot',
       'Apple___Cedar_apple_rust', 'Apple___healthy',
       'Blueberry___healthy', 'Cherry_(including_sour)___healthy',
       'Cherry_(including_sour)___Powdery_mildew',
       'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
       'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy',
       'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot',
       'Grape___Esca_(Black_Measles)', 'Grape___healthy',
       'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Invalid',
       'Orange___Haunglongbing_(Citrus_greening)',
       'Peach___Bacterial_spot', 'Peach___healthy',
       'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy',
       'Potato___Early_blight', 'Potato___healthy',
       'Potato___Late_blight', 'Raspberry___healthy', 'Rice___Brown_Spot',
       'Rice___Healthy', 'Rice___Leaf_Blast', 'Rice___Neck_Blast',
       'Soybean___healthy', 'Squash___Powdery_mildew',
       'Strawberry___healthy', 'Strawberry___L

# I am getting avoid to make a model from scratch coze of, It will take lots of time  for train

In [8]:
"""model = Sequential()
model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(2,2),padding="same",input_shape=(224,224,3),
                activation="relu")),
model.add(MaxPooling2D(2,2)),

model.add(Conv2D(64,kernel_size=(3,3),activation="relu")),
model.add(MaxPooling2D(2,2)),

model.add(Conv2D(64,kernel_size=(3,3),activation="relu")),
model.add(MaxPooling2D(2,2)),

model.add(Conv2D(64,kernel_size=(3,3),activation="relu")),
model.add(MaxPooling2D(2,2)),

model.add(Conv2D(64,kernel_size=(3,3),activation="relu")),
model.add(MaxPooling2D(2,2)),


model.add(Flatten()),


model.add(Dense(550,activation="relu")),
model.add(Dropout(0.3)),
model.add(Dense(460,activation='relu')),
model.add(Dropout(0.3)),
model.add(Dense(330,activation='relu')),
model.add(Dropout(0.3)),
model.add(Dense(230,activation='relu')),
model.add(Dropout(0.3)),
model.add(Dense(100,activation='relu')),
model.add(Dropout(0.3)),
model.add(Dense(100,activation='relu')),
model.add(Dropout(0.3)),
model.add(Dense(46,activation=tf.nn.softmax))"""

url = r"https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5"

model = Sequential()
model.add(hub.KerasLayer(url,trainable=False,input_shape=(224,224,3)))
model.add(Dense(46,activation="softmax"))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


# I'm going to use my fav... transfer learning

In [ ]:
url = r"https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5"

model = Sequential()
model.add(hub.KerasLayer(url,trainable=False,input_shape=(224,224,3)))
model.add(Dense(46,activation="softmax"))

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.categorical_crossentropy,
             metrics=["accuracy"])

In [10]:
class callback_end(tf.keras.callbacks.Callback):
    def on_batch_end(self,epochs,logs={}):
        if(logs.get("accuracy")>=0.97):
            print("Training stop......")
            self.model.stop_training=True
            
callback_data = callback_end()

In [11]:
model.fit(train_data,epochs=300,steps_per_epoch=len(test_data),
         validation_data=test_data,validation_steps=len(test_data),
         callbacks=[callback_data])
print(model.summary())

Epoch 1/300
602/602 [==============================] - 901s 1s/step - loss: 0.9327 - accuracy: 0.7837 - val_loss: 0.4675 - val_accuracy: 0.8755
Epoch 2/300
602/602 [==============================] - 874s 1s/step - loss: 0.3546 - accuracy: 0.9033 - val_loss: 0.3423 - val_accuracy: 0.9025
Epoch 3/300
Training stop......
602/602 [==============================] - 741s 1s/step - loss: 0.1180 - accuracy: 1.0000 - val_loss: 0.3406 - val_accuracy: 0.9007
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dense (Dense)               (None, 46)                47150     
                                                                 
Total params: 1,577,118
Trainable params: 47,150
Non-trainable params: 1,529,968
___________________________________________________

In [12]:
model.evaluate(test_data)

602/602 [==============================] - 552s 916ms/step - loss: 0.3399 - accuracy: 0.9022


[0.33987241983413696, 0.9022196531295776]

In [13]:
def preprocessing(filename,input_size=224):
    image = tf.io.read_file(filename)
    decode = tf.image.decode_image(image)
    resize = tf.image.resize(decode,size=[input_size,input_size])
    rescale = resize/255.
    return rescale

In [14]:
read_image = "Leaf_dataset/images_for_test/2.jpeg"
image = preprocessing(read_image)

In [15]:
pred_model = model.predict(tf.expand_dims(image,axis=0))
pred_class = class_name[(tf.argmax(pred_model[0]))]

1/1 [==============================] - 1s 1s/step


In [16]:
model.save("leaf_disease detector.keras")